Pada latihan-latihan sebelumnya pelatihan model berlangsung sebanyak jumlah epoch yang kita tentukan sebelumnya. Dan kalau kita amati, pada epoch-epoch terakhir akurasi dari model tidak meningkat lagi. Kasus lainnya misalnya kita memiliki 100 epoch, dan pada epoch ke 20 akurasi dari model sudah memenuhi target kita. 

Untunya kita dapat memberi tahu model kita untuk berhenti ketika telah mencapai akurasi tertentu sehingga proses pelatihan model menjadi lebih singkat. Bayangkan ketika waktu untuk eksekusi 1 epoch sebesar 6 detik. Untuk mengeksekusi 100 epoch berarti membutuhkan waktu selama 600 detik. Jika model kita telah mencapai target akurasi yang kita inginkan misalnya pada epoch ke 30, dan model otomatis berhenti melakukan pelatihan, maka kita bisa menghemat waktu eksekusi 70 epoch yaitu selama 420 detik. 

Kita dapat melakukan ini dengan menggunakan fungsi callback. Yup, fungsi callback membantu kita untuk memberi tahu model agar berhenti melakukan pelatihan ketika sudah mencapai target tertentu. Fungsi ini sangat berguna untuk menghemat waktu pelatihan dari model seperti yang dijelaskan pada paragraf sebelumnya.

Nah, langsung saja kita praktik menggunakan callback. Kita masih akan menggunakan dataset dan model dari latihan klasifikasi iris. Kode utuhnya seperti di bawah.

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

df = pd.read_csv('/content/sample_data/Iris.csv')

df = df.drop(columns='Id')

category = pd.get_dummies(df.Species)

new_df = pd.concat([df, category], axis=1)
new_df = new_df.drop(columns='Species')

dataset = new_df.values

X = dataset[:,0:4]
y = dataset[:,4:7]

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scale, y, test_size=0.3)

model = Sequential([    
                    Dense(64, activation='relu', input_shape=(4,)),    
                    Dense(64, activation='relu'),    
                    Dense(3, activation='sigmoid'),])

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



---



Selanjutnya kita buat kelas callback. Pada baris pertama lihatlah  bahwa kelas callback kita harus inherit `tf.keras.callbacks.Callback`. Selanjutnya kita buat fungsi yang paling penting yaitu `on_epoch_end()`. Fungsi inilah yang akan kita gunakan untuk memberitahu model agar berhenti melakukan pelatihan ketika telah mencapai target.

Selanjutnya kode `if(logs.get(‘accuracy’)>0.9)` sangat intuitif menunjukkan kalau kode itu memiliki arti “jika akurasi lebih besar dari 0.9 maka eksekusi perintah berikutnya”. Kita juga dapat menulis kode `print(“\nAkurasi telah mencapai >90%!”)` untuk ditampilkan ketika callback aktif.

Kode `self.model.stop_training = True` adalah kode yang memberi tahu model untuk menghentikan pelatihan. Setelah kelas ini dibuat kita buat objek dari kelas tersebut.

In [3]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nAkurasi telah mencapai > 90% !")
      self.model.stop_training = True

callbacks = myCallback()

Untuk menggunakan callback sangat gampang, kita hanya perlu menambahkan parameter 'callback' dan mengisi objek callback yang telah kita buat pada parameter tersebut.

In [4]:
model.fit(X_train, Y_train, epochs=50, callbacks=[callbacks])

Epoch 1/50
4/4 [==============================] - 1s 4ms/step - loss: 1.0404 - accuracy: 0.3048
Epoch 2/50
4/4 [==============================] - 0s 4ms/step - loss: 1.0077 - accuracy: 0.5048
Epoch 3/50
4/4 [==============================] - 0s 4ms/step - loss: 0.9757 - accuracy: 0.6381
Epoch 4/50
4/4 [==============================] - 0s 5ms/step - loss: 0.9450 - accuracy: 0.6476
Epoch 5/50
4/4 [==============================] - 0s 5ms/step - loss: 0.9123 - accuracy: 0.8095
Epoch 6/50
4/4 [==============================] - 0s 5ms/step - loss: 0.8786 - accuracy: 0.8571
Epoch 7/50
4/4 [==============================] - 0s 4ms/step - loss: 0.8455 - accuracy: 0.7524
Epoch 8/50
4/4 [==============================] - 0s 4ms/step - loss: 0.8101 - accuracy: 0.7048
Epoch 9/50
4/4 [==============================] - 0s 4ms/step - loss: 0.7768 - accuracy: 0.6857
Epoch 10/50
4/4 [==============================] - 0s 4ms/step - loss: 0.7421 - accuracy: 0.6762
Epoch 11/50
4/4 [======================

# Dari latihan di atas dapat dilihat model berhasil menghentikan pelatihan ketika telah mencapai akurasi yang kita tentukan pada epoch ke '35' dari '50'. Kita telah menghemat waktu eksekusi '15' epoch. Keren bukan?!.